# Update Model Package Approval Status

The pipeline that was executed created a Model Package version within the specified Model Package Group. Of particular note, the registration of the model/creation of the Model Package was done so with approval status as `PendingManualApproval`.

As part of SageMaker Pipelines, data scientists can register the model with approved/pending manual approval as part of the CI/CD workflow.

We can also approve the model using the SageMaker Studio UI or programmatically as shown below.

In [1]:
from botocore.exceptions import ClientError

import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [2]:
%store -r pipeline_name

In [3]:
print(pipeline_name)

BERT-pipeline-1708389284


In [4]:
%%time

import time
from pprint import pprint

executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)["PipelineExecutionSummaries"]
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

while pipeline_execution_status == "Executing":
    try:
        executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)["PipelineExecutionSummaries"]
        pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
    #        print('Executions for our pipeline...')
    #        print(pipeline_execution_status)
    except Exception as e:
        print("Please wait...")
        time.sleep(30)

pprint(executions_response)

Succeeded
[{'PipelineExecutionArn': 'arn:aws:sagemaker:eu-west-2:123137613716:pipeline/BERT-pipeline-1708389284/execution/2t51w088qj19',
  'PipelineExecutionDisplayName': 'execution-1708389293803',
  'PipelineExecutionStatus': 'Succeeded',
  'StartTime': datetime.datetime(2024, 2, 20, 0, 34, 53, 735000, tzinfo=tzlocal())}]
CPU times: user 11.7 ms, sys: 3.61 ms, total: 15.4 ms
Wall time: 161 ms


# List Pipeline Execution Steps


In [5]:
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

Succeeded


In [6]:
pipeline_execution_arn = executions_response[0]["PipelineExecutionArn"]
print(pipeline_execution_arn)

arn:aws:sagemaker:eu-west-2:123137613716:pipeline/BERT-pipeline-1708389284/execution/2t51w088qj19


In [7]:
from pprint import pprint

steps = sm.list_pipeline_execution_steps(PipelineExecutionArn=pipeline_execution_arn)

pprint(steps)

{'PipelineExecutionSteps': [{'AttemptCount': 1,
                             'EndTime': datetime.datetime(2024, 2, 20, 1, 14, 51, 806000, tzinfo=tzlocal()),
                             'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:eu-west-2:123137613716:model-package/BERT-Reviews-1708389285/1'}},
                             'StartTime': datetime.datetime(2024, 2, 20, 1, 14, 50, 887000, tzinfo=tzlocal()),
                             'StepName': 'RegisterModel',
                             'StepStatus': 'Succeeded'},
                            {'AttemptCount': 1,
                             'EndTime': datetime.datetime(2024, 2, 20, 1, 14, 52, 202000, tzinfo=tzlocal()),
                             'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:eu-west-2:123137613716:model/pipelines-2t51w088qj19-createmodel-zzste2vcyy'}},
                             'StartTime': datetime.datetime(2024, 2, 20, 1, 14, 50, 887000, tzinfo=tzlocal()),
                             'StepName'

# View Registered Model

In [8]:
for execution_step in steps["PipelineExecutionSteps"]:
    if execution_step["StepName"] == "RegisterModel":
        model_package_arn = execution_step["Metadata"]["RegisterModel"]["Arn"]
        break
print(model_package_arn)

arn:aws:sagemaker:eu-west-2:123137613716:model-package/BERT-Reviews-1708389285/1


In [9]:
model_package_update_response = sm.update_model_package(
    ModelPackageArn=model_package_arn,
    ModelApprovalStatus="Approved",  # Other options are Rejected and PendingManualApproval
)

# View Created Model

In [10]:
for execution_step in steps["PipelineExecutionSteps"]:
    if execution_step["StepName"] == "CreateModel":
        model_arn = execution_step["Metadata"]["Model"]["Arn"]
        break
print(model_arn)

model_name = model_arn.split("/")[-1]
print(model_name)

arn:aws:sagemaker:eu-west-2:123137613716:model/pipelines-2t51w088qj19-createmodel-zzste2vcyy
pipelines-2t51w088qj19-createmodel-zzste2vcyy


# Create Model Endpoint from Model Registry
More details here:  https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-deploy.html


In [11]:
import time

timestamp = int(time.time())

model_from_registry_name = "bert-model-from-registry-{}".format(timestamp)
print("Model from registry name : {}".format(model_from_registry_name))

model_registry_package_container = {
    "ModelPackageName": model_package_arn,
}

Model from registry name : bert-model-from-registry-1708393644


In [12]:
from pprint import pprint

create_model_from_registry_respose = sm.create_model(
    ModelName=model_from_registry_name, ExecutionRoleArn=role, PrimaryContainer=model_registry_package_container
)
pprint(create_model_from_registry_respose)

{'ModelArn': 'arn:aws:sagemaker:eu-west-2:123137613716:model/bert-model-from-registry-1708393644',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '97',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Tue, 20 Feb 2024 01:47:26 GMT',
                                      'x-amzn-requestid': 'c52f9ace-bae8-4e97-88ac-d62a728e88dc'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'c52f9ace-bae8-4e97-88ac-d62a728e88dc',
                      'RetryAttempts': 0}}


In [13]:
model_from_registry_arn = create_model_from_registry_respose["ModelArn"]
model_from_registry_arn

'arn:aws:sagemaker:eu-west-2:123137613716:model/bert-model-from-registry-1708393644'

In [14]:
endpoint_config_name = "bert-model-from-registry-epc-{}".format(timestamp)
print(endpoint_config_name)

create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m4.xlarge",
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

bert-model-from-registry-epc-1708393644


In [15]:
pipeline_endpoint_name = "bert-model-from-registry-ep-{}".format(timestamp)
print("EndpointName={}".format(pipeline_endpoint_name))

create_endpoint_response = sm.create_endpoint(
    EndpointName=pipeline_endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

EndpointName=bert-model-from-registry-ep-1708393644
arn:aws:sagemaker:eu-west-2:123137613716:endpoint/bert-model-from-registry-ep-1708393644


In [16]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker REST Endpoint</a></b>'.format(
            region, pipeline_endpoint_name
        )
    )
)

# _Wait Until the Endpoint is Deployed_

In [17]:
%%time

waiter = sm.get_waiter("endpoint_in_service")
waiter.wait(EndpointName=pipeline_endpoint_name)

# _Wait Until the Endpoint ^^ Above ^^ is Deployed_

# List All Artifacts

In [ ]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer

viz = LineageTableVisualizer(sagemaker.session.Session())

for execution_step in reversed(steps["PipelineExecutionSteps"]):
    print(execution_step)
    # We are doing this because there appears to be a bug of this LineageTableVisualizer handling the Processing Step
    if execution_step["StepName"] == "Processing":
        processing_job_name = execution_step["Metadata"]["ProcessingJob"]["Arn"].split("/")[-1]
        print(processing_job_name)
        display(viz.show(processing_job_name=processing_job_name))
    elif execution_step["StepName"] == "Train":
        training_job_name = execution_step["Metadata"]["TrainingJob"]["Arn"].split("/")[-1]
        print(training_job_name)
        display(viz.show(training_job_name=training_job_name))
    else:
        display(viz.show(pipeline_execution_step=execution_step))
        time.sleep(5)

# Test the Deployed Model

In [ ]:
import json
from sagemaker.tensorflow.model import TensorFlowPredictor
from sagemaker.serializers import JSONLinesSerializer
from sagemaker.deserializers import JSONLinesDeserializer

predictor = TensorFlowPredictor(
    endpoint_name=pipeline_endpoint_name,
    sagemaker_session=sess,
    model_name="saved_model",
    model_version=0,
    content_type="application/jsonlines",
    accept_type="application/jsonlines",
    serializer=JSONLinesSerializer(),
    deserializer=JSONLinesDeserializer(),
)

# Predict the `star_rating` with Ad Hoc `review_body` Samples

In [ ]:
inputs = [{"features": ["This is great!"]}, {"features": ["This is bad."]}]

predicted_classes = predictor.predict(inputs)

for predicted_class in predicted_classes:
    print("Predicted star_rating: {}".format(predicted_class))

In [ ]:
import csv

df_reviews = pd.read_csv(
    "./data/amazon_reviews_us_Digital_Software_v1_00.tsv.gz",
    delimiter="\t",
    quoting=csv.QUOTE_NONE,
    compression="gzip",
)

df_sample_reviews = df_reviews[["review_body", "star_rating"]].sample(n=50)
df_sample_reviews = df_sample_reviews.reset_index(drop=True)
df_sample_reviews.shape

In [ ]:
df_sample_reviews.head()

In [ ]:
import pandas as pd


def predict(review_body):
    inputs = [{"features": [review_body]}]
    predicted_classes = predictor.predict(inputs)
    return predicted_classes[0]["predicted_label"]


df_sample_reviews["predicted_class"] = df_sample_reviews["review_body"].map(predict)
df_sample_reviews.head(5)

# Release Resources

In [ ]:
# sm.delete_endpoint(
#      EndpointName=pipeline_endpoint_name
# )

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>

<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}